In [8]:
import pandas as pd
import numpy as np
import keras

# Set train,test and validate generators

In [7]:
datagen = keras.preprocessing.image.ImageDataGenerator()

In [81]:
train = datagen.flow_from_directory('../input/chest-xray-resized/train/', target_size=(300, 375), color_mode='grayscale',batch_size=64)
test = datagen.flow_from_directory('../input/chest-xray-resized/test/', target_size=(300, 375), color_mode='grayscale',batch_size=64)
val = datagen.flow_from_directory('../input/chest-xray-resized/val/', target_size=(300, 375), color_mode='grayscale',batch_size=64)
validation = datagen.flow_from_directory('../input/chest-xray-resized/validation/', target_size=(300, 375), color_mode='grayscale',batch_size=64)

Found 5216 images belonging to 3 classes.
Found 624 images belonging to 3 classes.
Found 16 images belonging to 3 classes.
Found 450 images belonging to 3 classes.


# Define CNN

In [61]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [75]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()
    
    model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='same', input_shape=(300, 375, 1)))
    model.add(LeakyReLU(0.1))
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    
    model.add(Flatten())
    
    model.add(Dense(units=256))
    model.add(LeakyReLU(0.1))
    
    model.add(Dropout(rate=0.5))
    
    model.add(Dense(units=3))
    model.add(Activation('softmax'))
    
    return model

In [76]:
model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 300, 375, 16)      160       
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 300, 375, 16)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 300, 375, 32)      4640      
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 300, 375, 32)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 150, 187, 32)      0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 150, 187, 32)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 150, 187, 32)      9248      
__________

In [77]:
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 10


# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 10-way classification
    optimizer=keras.optimizers.adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

In [80]:
last_finished_epoch=None
for i in range(len(train)):
    model.fit(
        train[i][0], train[i][1],  # prepared data
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
                   #LrHistory(), 
                   # keras_utils.TqdmProgressCallback(),
                   #keras.utils.ModelSaveCallback(model_filename)
                  ],
        validation_data=(test[i][0], test[i][1]),
        shuffle=True,
        verbose=1,
        initial_epoch=last_finished_epoch or 0
    )

Train on 64 samples, validate on 64 samples
Epoch 1/10
64/64 [==============================] - 466s 7s/step - loss: 9.9809 - acc: 0.3281 - val_loss: 10.3257 - val_acc: 0.3594
Epoch 2/10
64/64 [==============================] - 296s 5s/step - loss: 10.0738 - acc: 0.3750 - val_loss: 10.3257 - val_acc: 0.3594
Epoch 3/10
64/64 [==============================] - 659s 10s/step - loss: 11.0812 - acc: 0.3125 - val_loss: 10.3257 - val_acc: 0.3594
Epoch 4/10
32/64 [==============>...............] - ETA: 3:28 - loss: 12.0886 - acc: 0.2500

KeyboardInterrupt: 

In [69]:
print(X[100][1])

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
